In [43]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from preprocessing_helpers import *
from Basics import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [3]:
# split test-train set
indx = np.arange(y.shape[0])
np.random.shuffle( indx )
ind_max = int(y.shape[0] * (0.8))

trainY , trainF = y[indx[:ind_max]] , tX[indx[:ind_max]]
testY , testF = y[indx[ind_max:]] , tX[indx[ind_max:]]

In [4]:
####### Preprocessing ######################################################################

In [5]:
# normalize
trainF, testF = standardize(trainF, testF)
trainF = np.insert(trainF ,0 , np.ones(trainF.shape[0]),axis = 1) # add bias
testF = np.insert(testF ,0 , np.ones(testF.shape[0]),axis = 1) # add bias

In [6]:
# replace -999
trainF, testF = replace_missing_values(trainF,testF, 'mode')


In [7]:
# replace outliers
#trainF = outliers_removal(trainF)
#testF = outliers_removal(testF)

In [8]:
# dataset balencing
#trainF , trainY =  under_over(trainF, trainY, alpha=1, upsample=True, middle=True, gaussian=False, std=0.1, downsample=False)

In [68]:
# feature selection expansion
trainF_sel = select_and_expand_f_logistic(trainF)
testF_sel = select_and_expand_f_logistic(testF)

trainF_sel.shape

TEMP -> 495
TEMP -> 495


(200000, 496)

In [69]:
###### Training/Testing ######################################################################

In [70]:
#w_initial = np.random.rand(trainF_sel.shape[1])
#w,_ = least_squares_GD( trainY,trainF_sel ,w_initial,2000,0.01)
#get_accuracy(w , testY ,testF_sel )

In [71]:
#w_initial = np.random.rand(trainF_sel.shape[1])
#w,_ = least_squares(trainY,trainF_sel)
#get_accuracy(w , testY ,testF_sel )

In [72]:
#w = np.random.rand(trainF_sel.shape[1])
#w, losses, _, _, _, _ = foo_logistic_regression(trainY, trainF_sel, w, 1000, 0.01, testY, testF_sel, 0)

In [77]:
w = np.random.rand(trainF_sel.shape[1])


w, _ = logistic_regression(trainY, trainF_sel, w, 500 ,0.07/y.shape[0])

w_500 = w
print( "acc 500  : ",get_accuracy_logistic(w_500 , testY ,testF_sel))

w, _ = logistic_regression(trainY, trainF_sel, w_500, 500 ,0.07/y.shape[0])

w_1000 = w
print( "acc 1000 : ",get_accuracy_logistic(w_1000 , testY ,testF_sel))

w, _ = logistic_regression(trainY, trainF_sel, w_1000, 1000 ,0.07/y.shape[0])

w_2000 = w
print( "acc 2000 : ",get_accuracy_logistic(w_2000 , testY ,testF_sel))

acc 500  :  0.7768
acc 1000 :  0.7973
acc 2000 :  0.8091


In [78]:
w, _ = logistic_regression(trainY, trainF_sel, w_2000, 1000 ,0.07/y.shape[0])

w_3000 = w
print( "acc 3000 : ",get_accuracy_logistic(w_3000 , testY ,testF_sel))

acc 3000 :  0.81372


In [79]:
w, _ = logistic_regression(trainY, trainF_sel, w_3000, 1000 ,0.07/y.shape[0])

w_4000 = w
print( "acc 4000 : ",get_accuracy_logistic(w_4000 , testY ,testF_sel))

acc 4000 :  0.81686


In [80]:
w, _ = logistic_regression(trainY, trainF_sel, w_4000, 1000 ,0.07/y.shape[0])

w_5000 = w
print( "acc 5000 : ",get_accuracy_logistic(w_5000 , testY ,testF_sel))

acc 5000 :  0.81822


In [81]:
w, _ = logistic_regression(trainY, trainF_sel, w_5000, 2000 ,0.07/y.shape[0])

w_7000 = w
print( "acc 7000 : ",get_accuracy_logistic(w_7000 , testY ,testF_sel))

acc 5000 :  0.81944


In [82]:
w, _ = logistic_regression(trainY, trainF_sel, w_7000, 2000 ,0.07/y.shape[0])

w_9000 = w
print( "acc 9000 : ",get_accuracy_logistic(w_9000 , testY ,testF_sel))

acc 9000 :  0.82056


In [83]:
w, _ = logistic_regression(trainY, trainF_sel, w_9000, 1000 ,0.07/y.shape[0])

w_10000 = w
print( "acc 10000 : ",get_accuracy_logistic(w_10000 , testY ,testF_sel))

acc 10000 :  0.82042


In [84]:
w, _ = logistic_regression(trainY, trainF_sel, w_10000, 5000 ,0.07/y.shape[0])

w_15000 = w
print( "acc 15000 : ",get_accuracy_logistic(w_15000 , testY ,testF_sel))

acc 10000 :  0.8211


In [85]:
w, _ = logistic_regression(trainY, trainF_sel, w_15000, 5000 ,0.07/y.shape[0])

w_20000 = w
print( "acc 20000 : ",get_accuracy_logistic(w_20000 , testY ,testF_sel))

acc 20000 :  0.82138


In [87]:
w_20000 = np.array([ 3.11501280e-01,  8.57036622e-01, -2.23400033e-01,  4.01059408e-01,
        6.85446414e-01,  2.12913059e-01,  1.97085042e-01, -1.42896100e-01,
        7.44002112e-01, -3.08966857e-02, -2.58304669e-01, -5.21373659e-02,
        5.30324738e-01,  6.91035535e-01,  8.70069751e-01,  1.33219944e-02,
        1.16373363e-01, -7.61140873e-02,  1.04019901e-02,  7.28748647e-02,
       -2.14867329e-01, -3.45676915e-02, -6.08520155e-02, -2.88366447e-01,
        4.58369286e-01, -5.51562569e-02,  4.06153463e-02,  5.41535423e-02,
       -1.00128313e-01,  1.05435492e-01, -3.61010994e-03, -1.14454220e-02,
       -2.00830173e-01,  3.12738840e-01, -5.09308702e-01,  6.37972232e-02,
       -2.94941575e-01,  3.44023286e-02, -1.24249025e+00, -6.62026562e-02,
       -3.45439838e-01,  1.64861228e-01, -1.16454989e-01,  3.21989039e-03,
       -2.94262716e-01, -3.87429873e-02, -5.95287904e-02, -4.43247628e-01,
        2.86592251e-02, -1.29711277e-02,  8.51827727e-02, -1.93776451e-02,
       -7.61475784e-02,  3.81530626e-02, -1.70422002e-01, -1.05239559e-02,
       -1.23092603e-02, -2.37622651e-02, -3.38266150e-02, -1.60958334e-01,
        6.93318707e-01, -5.85611899e-04, -3.22179421e-01, -5.33768927e-01,
        1.04129195e-01, -8.51218545e-02, -7.07655332e-02, -3.48249124e-01,
        8.75793637e-02, -4.94561616e-01, -1.40757404e-02,  1.60434482e-01,
       -6.47355773e-02,  2.14550075e-01, -2.23761195e-02, -2.34136340e-03,
        2.87631289e-01,  2.20476397e-03,  1.99578433e-04,  1.50734644e-01,
        1.14286532e-02,  9.02464001e-02,  1.82635481e-01,  3.31861033e-01,
        1.98039511e-02, -6.95779745e-03,  2.20957004e-01,  2.18463790e-02,
       -4.29584796e-02, -1.48415914e-01,  1.03765092e-01, -1.30984280e-01,
        4.96770645e-01,  6.87645162e-03,  5.20010100e-02, -9.45830722e-01,
        7.23888259e-02, -8.78490915e-02, -9.72246508e-03,  4.99797969e-01,
       -5.69189637e-02, -8.70666130e-02,  3.86274333e-02,  6.60458359e-02,
       -2.34390635e-02, -6.06970861e-02, -1.93827062e-02,  1.83047377e-01,
       -7.27500725e-03, -7.80573897e-02, -1.26569346e-01, -9.39191703e-03,
        5.55307263e-02, -9.60973496e-02,  2.16449730e-01,  4.09006614e-02,
        1.67388657e-01, -9.39643442e-02, -5.34935451e-03, -1.52708733e-01,
       -6.42129987e-02, -1.53093384e-01,  3.20172073e-01,  1.66676116e-01,
       -3.55267396e-01,  1.39976516e-01,  1.16506374e-01,  1.80499597e-01,
        1.32878319e-01, -3.16542687e-02, -2.72891385e-02,  2.59265458e-02,
       -3.52955935e-02,  6.94954445e-02, -1.27556203e-01,  1.10080627e-02,
        8.64412399e-02,  3.02941300e-02,  2.12566028e-01,  6.21010528e-02,
       -4.95938831e-02,  2.09770570e-01,  1.46837553e-01, -2.89651638e-02,
        4.05558651e-01, -2.29444409e-01,  4.69382805e-01, -1.35991131e-01,
       -3.03600692e-01, -8.16740452e-02,  1.67062435e-02,  2.02647573e-01,
       -8.68711462e-02,  2.95739598e-01, -5.88114365e-02,  8.75766766e-02,
       -2.60943387e-02, -2.76103262e-01, -1.38109822e-01, -6.50591901e-02,
        1.28203592e-01, -1.37599609e-02, -1.39858924e-01, -2.74108871e-01,
        3.34810642e-03,  2.02487350e-02,  8.49483849e-03,  3.02615424e-01,
        8.12743359e-02, -1.45988277e-01,  2.06282046e-01,  1.14870482e-01,
        5.75459344e-01,  4.62901404e-01,  4.30359309e-02, -4.35152192e-01,
        1.70024150e-02,  2.33450781e-01,  6.24245302e-01,  3.05485996e-01,
       -1.72404388e-01,  9.96993794e-02,  2.51250946e-01,  1.49479620e-01,
        1.23052093e-01,  1.06628894e-01, -3.29591137e-02,  2.65078304e-02,
       -1.47481194e-01,  4.62981429e-02,  9.97751726e-02,  7.64565147e-02,
       -1.69666908e-01, -5.65559366e-02,  1.73417845e-01,  1.38354223e-01,
        4.57061135e-03,  8.65253388e-02,  2.88734037e-03, -6.63726840e-01,
        1.39039325e-01,  2.77781792e-02,  1.05315314e-01,  2.90285854e-01,
        1.33791096e-02,  3.98995162e-02,  7.96168650e-02,  1.58288488e-03,
        2.23068099e-02,  1.63020438e-01, -6.00248305e-03, -5.83424385e-02,
        1.97048232e-01,  1.99414993e-01,  7.21069137e-02,  2.68094886e-02,
        1.59227056e-01, -4.03766331e-02, -1.38725256e-02,  1.51124824e-01,
        2.89662799e-02,  1.32267895e-02,  4.41253579e-01, -1.74933301e-01,
       -4.30875372e-01,  2.71188305e-01,  1.22282462e-01, -7.49684574e-05,
        6.46162492e-03,  1.25715140e-01, -5.66570401e-03,  4.74142180e-02,
       -5.79566631e-01,  4.76775953e-02,  1.17841048e-01, -3.38696523e-01,
        3.32464790e-01,  3.67564346e-03,  1.09972656e-01, -1.81359744e-01,
        1.50282329e-02,  3.09961768e-02, -3.54494628e-01,  2.35743733e-02,
        6.55685576e-02, -1.67293667e-03, -1.31254100e-01, -4.45941614e-03,
       -3.49088148e-02, -1.90488153e-02, -2.58984141e-03, -7.62206060e-02,
       -5.81193900e-03, -7.44948626e-03, -7.98135436e-02,  3.13361387e-03,
       -7.28694303e-02,  3.99466384e-02,  1.30563190e-02,  9.66798188e-03,
       -3.19244616e-02, -2.38682002e-03,  1.64922408e-02, -1.33173435e-03,
       -7.64091580e-02,  7.56439358e-02,  4.24835548e-02,  6.81177323e-03,
       -1.73792343e-01, -2.64243118e-01, -2.34218531e-01, -4.16781366e-01,
       -2.06579244e-01, -3.27198147e-01, -3.32570090e-01, -2.52405526e-01,
        2.57024726e-01,  1.02611612e-01, -2.33808569e-01, -1.87096339e-01,
       -2.61489366e-01, -1.78122745e-01, -3.60486065e-01, -1.32454725e-02,
       -3.02660725e-01,  4.19650816e-01, -1.42977927e-02,  1.38510516e-01,
       -3.53668608e-02,  1.67414039e-01, -1.18573108e-02, -1.20115749e-02,
        9.58028365e-03, -1.64733576e-02, -1.65705442e-02, -2.99241855e-02,
       -3.13848321e-02,  1.74232099e-02,  6.14041174e-02, -8.44954144e-02,
        2.81546036e-02, -6.67254660e-02,  5.40240868e-02, -1.75327465e-02,
       -3.07645169e-02, -1.46290243e-01,  8.52893818e-02,  3.62639525e-02,
       -1.79884833e-01,  1.32560951e-02,  1.13606561e-02, -2.19811631e-01,
       -2.52967978e-02, -1.16545233e-02,  2.01365661e-01,  3.97759054e-03,
        4.03384961e-02,  2.32650820e-02, -8.90398798e-02, -4.06542329e-03,
        2.81045558e-03,  3.98479554e-02, -9.76207072e-03,  1.95033472e-02,
       -1.33951971e-01, -2.06971503e-01,  6.78751286e-02,  1.55956168e-02,
        1.41626903e-03,  8.68713485e-02,  2.59877939e-02,  3.70310749e-02,
       -2.89856602e-02, -1.47199808e-02, -6.89415185e-02,  3.06977518e-01,
        1.49389961e-01, -3.51479185e-02, -2.54578049e-02,  1.18549233e-01,
       -2.86162640e-02,  1.80306300e-02, -3.14326313e-01,  1.99399817e-02,
        5.22789833e-02,  6.19125234e-02,  1.35722533e-01,  6.00588893e-02,
        4.90124220e-02,  1.05976675e-02, -6.25210447e-02,  7.61872946e-03,
       -7.46748775e-04,  7.24434577e-03,  2.06536228e-02,  3.99332366e-02,
       -6.78004955e-02, -4.80097944e-02,  6.73473200e-02,  4.12246466e-02,
       -3.41541330e-01, -1.24405844e-02,  4.40583071e-02,  4.72212740e-01,
       -1.07692606e-02,  3.04460031e-02,  6.69127429e-03,  1.93186830e-02,
        7.24629155e-03,  5.90144041e-02, -5.34580342e-02, -1.22233680e-02,
        5.74302560e-02,  8.59535684e-02,  3.06199355e-03,  1.29738160e-01,
        3.00691220e-03,  8.76726973e-02,  1.59428924e-02, -2.04642305e-04,
        2.42098143e-02,  2.23305089e-02, -5.96939110e-03,  6.60441136e-02,
        1.44652003e-01, -1.31161211e-03,  1.90874789e-02,  6.50633675e-02,
        1.83540227e-02,  1.99108620e-02,  1.55206295e-01,  2.24218257e-02,
        9.02895124e-02,  6.78739714e-02,  3.85495821e-04, -3.99866668e-02,
       -1.14307463e-02,  6.76757484e-02,  9.84893469e-02,  1.44210740e-03,
        1.02975404e-01, -1.18393061e-01, -3.30516660e-02,  7.78064526e-02,
        8.04769598e-02, -3.98297732e-01,  1.20090651e-02, -7.82826518e-03,
       -5.93951371e-03, -1.85285665e-02,  8.03006947e-02,  5.14542667e-02,
       -2.10653495e-02,  5.13534544e-03, -7.98688091e-03, -5.94415686e-02,
       -1.10043291e-02,  2.23334253e-01, -3.05768697e-03, -6.29698002e-03,
        7.46581967e-03,  9.30834890e-03,  2.71399941e-02,  5.25668567e-02,
       -1.28180692e-02, -1.95272678e-02,  2.26465834e-02, -1.33106077e-03,
        2.27695050e-03,  1.58460046e-01,  1.18513595e-02, -1.06267886e-02,
       -7.12239639e-02, -9.89205735e-02, -6.00362283e-02, -1.97761076e-02,
        3.36600461e-02, -1.08367964e-01, -7.99829961e-02,  7.50536387e-02,
        1.83955719e-01,  1.36026977e-03,  1.77875900e-02,  8.74233459e-02,
        5.65154041e-02,  7.05786989e-03,  3.32276524e-02,  2.00988828e-02,
       -2.74442618e-02, -3.29043638e-03, -2.99915618e-01, -1.31312209e-02,
        3.39761457e-02,  9.44510292e-02,  1.72857144e-03,  7.02470474e-02,
        2.08462414e-01, -5.40165416e-05, -1.18861326e-02, -2.84283768e-01,
        4.57188469e-02,  2.61322623e-02,  5.02211895e-02,  9.67873717e-03,
        1.72771589e-03,  6.01367098e-02,  7.54717783e-02, -1.70755934e-01,
       -1.26633175e-01, -2.98348085e-03,  6.02717988e-02,  1.05596315e-03,
        5.15512859e-02,  2.20591050e-02,  1.37681175e-01,  3.69620210e-01,
       -4.62199783e-03, -5.20531571e-02,  1.81311692e-01, -5.19823268e-03,
        2.15089778e-01,  1.39659920e-02, -2.65059296e-02,  2.50123428e-02,
        3.33330462e-02,  8.92841477e-02, -2.38345439e-02,  4.98017892e-02,
        7.34587027e-02,  1.94599358e-01,  3.76055617e-01, -4.52111947e-03,
       -7.11960872e-02, -1.20375389e-02,  8.42577342e-02, -1.49577133e-01])

In [88]:
print( "acc 20000 : ",get_accuracy_logistic(w_20000 , testY ,testF_sel))

acc 20000 :  0.82138


In [ ]:
w, _ = logistic_regression(trainY, trainF_sel, w_15000, 10000 ,0.07/y.shape[0])

w_30000 = w
print( "acc 30000 : ",get_accuracy_logistic(w_30000 , testY ,testF_sel))